# Upload data to GCS Storage

In [2]:
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'gcs-key.json'

In [3]:
from google.cloud import storage

In [4]:
def upload_partitioned_dataset_skip_existing(local_root, bucket_name, gcs_root):
    client = storage.Client()
    bucket = client.bucket(bucket_name)

    for root, dirs, files in os.walk(local_root):
        for file in files:
            local_path = os.path.join(root, file)
            relative_path = os.path.relpath(local_path, local_root)
            gcs_path = os.path.join(gcs_root, relative_path).replace("\\", "/")

            blob = bucket.blob(gcs_path)

            if blob.exists():
                print(f"⏩ Skipped existing file: gs://{bucket_name}/{gcs_path}")
            else:
                blob.upload_from_filename(local_path)
                print(f"✅ Uploaded {local_path} to gs://{bucket_name}/{gcs_path}")

In [5]:
import pandas as pd
import requests

In [6]:
df = pd.read_parquet("gs://bernacho-ecobici-datahub/partitioned_historical_data", filters=[("year",'>=',2022),("year",'<=',2024)])

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39503816 entries, 0 to 39503815
Data columns (total 15 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   Genero_Usuario         object        
 1   Edad_Usuario           Int64         
 2   Bici                   object        
 3   Ciclo_Estacion_Retiro  object        
 4   Fecha_Retiro           datetime64[ns]
 5   Hora_Retiro            object        
 6   Ciclo_Estacion_Arribo  object        
 7   Fecha_Arribo           datetime64[ns]
 8   Hora_Arribo            object        
 9   date_start             datetime64[ns]
 10  date_end               datetime64[ns]
 11  duration               int32         
 12  file                   object        
 13  year                   category      
 14  month                  category      
dtypes: Int64(1), category(2), datetime64[ns](4), int32(1), object(7)
memory usage: 3.8+ GB


In [8]:
df['year'] = df['year'].astype(int)
df['month'] = df['month'].astype(int)

In [9]:
df[['year','month']].value_counts().sort_index()

year  month
2022  1         393115
      2         434262
      3         557931
      4         523923
      5         594329
      6         527685
      7         405082
      8         358892
      9         203064
      10        232276
      11        422208
      12        431484
2023  1         586402
      2         664948
      3         817979
      4         813608
      5         945162
      6        1009992
      7         999706
      8        1109194
      9        1189627
      10       1401777
      11       1410644
      12       1228319
2024  1        1558095
      2        1699393
      3        1829741
      4        1961618
      5        2053545
      6        1893513
      7        1825635
      8        1891298
      9        1852147
      10       2054419
      11       1942324
      12       1680479
Name: count, dtype: int64

In [10]:
df[(df.year==2022) & (df.month<9)]['Ciclo_Estacion_Retiro'].value_counts().loc[["2"]].head(20)

Ciclo_Estacion_Retiro
2    9249
Name: count, dtype: int64

In [12]:
df[df.year>=2023]['Ciclo_Estacion_Retiro'].value_counts().loc[["002"]].head(20)

Ciclo_Estacion_Retiro
002    76436
Name: count, dtype: int64

In [13]:
df[df['Ciclo_Estacion_Retiro'].isin(["2","002"])].groupby(['year','month']).size().sort_index()

year  month
2022  1        1066
      2        1146
      3        1302
      4        1242
      5        1433
      6         930
      7        1103
      8        1027
      9         489
      10        389
      11       2020
      12       1834
2023  1        2381
      2        2412
      3        2591
      4        2515
      5        2930
      6        2483
      7        2497
      8        2925
      9        2685
      10       3172
      11       3380
      12       2605
2024  1        3594
      2        3714
      3        3642
      4        4022
      5        4141
      6        3872
      7        3895
      8        3653
      9        3479
      10       3543
      11       3397
      12       2908
dtype: int64

In [14]:
df[df.year==2024]['Ciclo_Estacion_Retiro'].value_counts().head(20)

Ciclo_Estacion_Retiro
271-272    245197
027        133176
064        126428
237-238    113104
031        105518
014        105113
208        105038
107-108    104051
018        103340
192-193    102105
028        101311
001        101227
041        101129
158-159     97445
548         96410
052         94746
555         93496
194         93294
242         89529
038         88811
Name: count, dtype: int64

In [23]:
df[df['Ciclo_Estacion_Retiro']=="2"]['Ciclo_Estacion_Arribo'].value_counts().head(10)

Ciclo_Estacion_Arribo
2      366
1      319
16     312
35     294
34     293
32     292
9      258
24     251
103    199
27     177
Name: count, dtype: int64

In [24]:
df[df['Ciclo_Estacion_Retiro']=="002"]['Ciclo_Estacion_Arribo'].value_counts().head(10)

Ciclo_Estacion_Arribo
002        2629
001        2264
271-272    2181
016        1806
027        1659
032        1428
014        1344
056        1175
007        1096
019        1006
Name: count, dtype: int64

In [16]:
df[df['Ciclo_Estacion_Retiro']=="271-272"].groupby(['year','month']).size().sort_index()

year  month
2022  10        1434
      11        7829
      12        9576
2023  1        10306
      2         8537
      3         9231
      4         9789
      5        13228
      6        13968
      7        13723
      8        13208
      9        15884
      10       17622
      11       17139
      12       17132
2024  1        18905
      2        16639
      3        18124
      4        19293
      5        24203
      6        22869
      7        22951
      8        22421
      9        22284
      10       24460
      11       20731
      12       12317
dtype: int64

In [27]:
a.sort_index()

Ciclo_Estacion_Retiro
1      4710
10     2215
100     588
102     645
103    2034
       ... 
95     1279
96     1896
97      766
98     1054
99     1284
Length: 475, dtype: int64

In [34]:
url = "https://gbfs.mex.lyftbikes.com/gbfs/gbfs.json"
response = requests.get(url,timeout=2)
api_urls = {x['name']: x['url'] for x in response.json()['data']['en']['feeds']}
response = requests.get(api_urls['station_information'],timeout=3.1)
stations = pd.DataFrame.from_dict(response.json()['data']['stations'])
stations['station_id'] = stations['station_id'].str.lstrip("0")
# stations['short_name'] = stations['short_name'].str.lstrip("0")
stations.set_index("station_id",inplace=True)
stations.shape

(677, 11)

In [35]:
stations.loc["2"]

KeyError: '2'

In [64]:
stations.to_csv("new_stations.csv",index=True)

In [62]:
api_urls

{'station_status': 'https://gbfs.mex.lyftbikes.com/gbfs/en/station_status.json',
 'station_information': 'https://gbfs.mex.lyftbikes.com/gbfs/en/station_information.json',
 'system_alerts': 'https://gbfs.mex.lyftbikes.com/gbfs/en/system_alerts.json',
 'free_bike_status': 'https://gbfs.mex.lyftbikes.com/gbfs/en/free_bike_status.json',
 'system_information': 'https://gbfs.mex.lyftbikes.com/gbfs/en/system_information.json'}

In [ ]:
stations.station_id.astype(int).sort_values()

short_name
710      1
407      5
428      6
427      7
443      8
      ... 
511    697
682    698
681    699
711    700
524    702
Name: station_id, Length: 677, dtype: int64

In [48]:
a.to_frame("trip_count").assign(station_name=lambda x: x.index.map(stations.set_index('station_id').name)).loc[lambda x: x.station_name.isna()]

,trip_count,station_name
Ciclo_Estacion_Retiro,,
308,663,NaN
13,787,NaN
248,900,NaN
2,930,NaN
351,1019,NaN
274,1305,NaN
3,1557,NaN
310,1662,NaN
4,1707,NaN


In [54]:
pd.set_option('display.max_rows', 500)

In [57]:
stations[stations.station_id.str[:1].str.contains("2")]

,station_id,external_id,name,lat,lon,rental_methods,capacity,electric_bike_surcharge_waiver,is_charging,eightd_has_key_dispenser,has_kiosk
short_name,,,,,,,,,,,
412,20,8c4eceec-211c-4064-b923-97c71f09b1a7,CE-412 Parroquia-Avenida Universidad,19.368107,-99.164376,"[CREDITCARD, KEY]",34,False,False,False,True
381,21,616c0715-1a8e-492b-85cb-452f062ca42b,CE-381 San Lorenzo-Manzanas,19.375543,-99.178011,"[CREDITCARD, KEY]",23,False,False,False,True
332,22,f9827216-5cfa-4eb1-a912-096b9545fa83,CE-332 Calle 27-Avenida 3,19.385494,-99.183744,"[CREDITCARD, KEY]",23,False,False,False,True
399,23,ef08f966-4d87-4fdc-b783-287605e9c117,CE-399 Heriberto Frías-San Lorenzo,19.372469,-99.164538,"[CREDITCARD, KEY]",35,False,False,False,True
377,24,78a8e829-3e75-40fd-9543-3ca0cbc15747,CE-377 Heriberto Frías-Miguel Laurent,19.375397,-99.163778,"[CREDITCARD, KEY]",35,False,False,False,True
360,25,287e590f-e27b-4211-8fb5-9b11bd2fd0cd,CE-360 Heriberto Frías-Matias Romero,19.379435,-99.162794,"[CREDITCARD, KEY]",23,False,False,False,True
362,26,d353123d-74b4-405b-8257-a61eb1d771ee,CE-362 Augusto Rodin-Alberto Balderas,19.382123,-99.181091,"[CREDITCARD, KEY]",19,False,False,False,True
365,27,df57391d-94b4-497a-86f0-0681ef80704a,CE-365 Holbein-Avenida Revolución,19.381123,-99.186670,"[CREDITCARD, KEY]",21,False,False,False,True
424,28,4129eb20-73b3-4e3b-b473-a939ab14156f,CE-424 Amores-José María Rico (Eje 8),19.367103,-99.170581,"[CREDITCARD, KEY]",11,False,False,False,True


In [38]:
stations.index.sort_values()

Index(['1', '10', '100', '102', '103', '104', '105', '106', '107-108', '109',
       ...
       '90', '91', '92', '93', '94', '95', '96', '97', '98', '99'],
      dtype='object', name='short_name', length=677)

In [42]:
stations[stations['station_id']=='271']

,station_id,external_id,name,lat,lon,rental_methods,capacity,electric_bike_surcharge_waiver,is_charging,eightd_has_key_dispenser,has_kiosk
short_name,,,,,,,,,,,
141,271,da798f39-7b3c-400c-9c2d-450360eb82b4,CE-141 Zacatecas-Luis Cabrera,19.415905,-99.15954,"[CREDITCARD, KEY]",31,False,False,False,True


In [5]:
data_path = "data/partitioned_historical_data"

In [7]:
upload_partitioned_dataset_skip_existing(
    local_root=data_path,
    bucket_name='bernacho-ecobici-datahub',
    gcs_root='partitioned_historical_data'
)


✅ Uploaded data/partitioned_historical_data\year=2019\month=1\786eac27676e413bb2fa0bbe0b2e6579-0.parquet to gs://bernacho-ecobici-datahub/ecobici_partitioned_data/year=2019/month=1/786eac27676e413bb2fa0bbe0b2e6579-0.parquet
✅ Uploaded data/partitioned_historical_data\year=2019\month=10\b992a1416dde40468076a636d0a80afd-0.parquet to gs://bernacho-ecobici-datahub/ecobici_partitioned_data/year=2019/month=10/b992a1416dde40468076a636d0a80afd-0.parquet
✅ Uploaded data/partitioned_historical_data\year=2019\month=11\9ea28ccbdfd74f409e90254c51d1b2aa-0.parquet to gs://bernacho-ecobici-datahub/ecobici_partitioned_data/year=2019/month=11/9ea28ccbdfd74f409e90254c51d1b2aa-0.parquet
✅ Uploaded data/partitioned_historical_data\year=2019\month=12\775d14e8e5a74e0791c905939c748aa5-0.parquet to gs://bernacho-ecobici-datahub/ecobici_partitioned_data/year=2019/month=12/775d14e8e5a74e0791c905939c748aa5-0.parquet
✅ Uploaded data/partitioned_historical_data\year=2019\month=2\76edf059dc134ad581276ca077913aba-0.p